In [1]:
import h2o
from h2o.automl import H2OAutoML

from pyspark.sql import SparkSession
from pysparkling import *

In [2]:
# input_1 : 
Window_size = 5
train_folder = 'input_1/train'
valid_folder = 'input_1/valid'
n_fold = 5
n_digits = 4
seed = 1
max_runtime_secs = 1000
response = "label"

# input_2 : 
Window_size = 10
#train_folder = 'input_2/train'
#valid_folder = 'input_2/valid'
#n_fold = 5
#n_digits = 3
#seed = 2
#max_runtime_secs = 1500
#response = "label"

# train_folder = 'small/small_train'
# valid_folder = 'small/small_valid'
# n_fold = 5
# n_digits = 4
# seed = 1
# max_runtime_secs = 1000
# response = "label"


In [3]:
ss = SparkSession.builder.config("spark.executor.memory", "12g")\
    .config("spark.driver.memory", "12g")\
    .config("spark.executor.cores", 6)\
    .config('spark.executor.instances', 5).getOrCreate()

ss.sparkContext.setLogLevel('OFF')
hc = H2OContext.getOrCreate()


Connecting to H2O server at http://192.168.0.11:54323 ... successful.


H2O_cluster_uptime:,09 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,1 month and 5 days
H2O_cluster_name:,sparkling-water-yuefeiwang_local-1615078077297
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,10.54 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,6
H2O_cluster_status:,"locked, healthy"



Sparkling Water Context:
 * Sparkling Water Version: 3.32.0.4-1-3.0
 * H2O name: sparkling-water-yuefeiwang_local-1615078077297
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (0,192.168.0.11,54321)
  ------------------------

  Open H2O Flow in browser: http://192.168.0.11:54323 (CMD + click in Mac OSX)

    


In [ ]:
# mute the progress bar
h2o.no_progress()

# Q1

In [4]:
# create spark dataframe
train_df = ss.read.parquet(train_folder).repartition(8).cache()
valid_df = ss.read.parquet(valid_folder).repartition(8).cache()


In [7]:

# convert spark dataframe to h2oFrame
train_h2o = hc.asH2OFrame(train_df, "train")
valid_h2o = hc.asH2OFrame(valid_df, "valid")

In [9]:
# convert label column to categorical datatype
train_h2o['label'] = train_h2o['label'].asfactor()
valid_h2o['label'] = valid_h2o['label'].asfactor()

# Q2

In [12]:
predictors = train_h2o.names[:]
response = 'label'
predictors.remove(response)

# Q3

In [15]:
from h2o.automl import H2OAutoML
model_automl = H2OAutoML(max_runtime_secs=max_runtime_secs, seed = seed, nfolds = n_fold) # seed : Set a seed for reproducibility
model_automl.train(x=predictors,
                   y=response,
                   training_frame=train_h2o,
                   validation_frame=valid_h2o)

AutoML progress: |
16:53:41.113: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

████████████████████████████████████████████████████████| 100%


In [27]:
print(model_automl.leaderboard)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
XGBoost_2_AutoML_20210306_165341,0.958249,0.50625,0.908446,0.111932,0.398415,0.158735
StackedEnsemble_BestOfFamily_AutoML_20210306_165341,0.958211,0.243636,0.908049,0.112815,0.266111,0.0708151
StackedEnsemble_AllModels_AutoML_20210306_165341,0.957537,0.239183,0.90503,0.112147,0.264056,0.0697256
XGBoost_1_AutoML_20210306_165341,0.893553,0.55673,0.747795,0.187428,0.428044,0.183222
XGBoost_3_AutoML_20210306_165341,0.792847,0.598657,0.558004,0.276257,0.451242,0.203619
GLM_1_AutoML_20210306_165341,0.62529,0.58957,0.363773,0.404796,0.447493,0.20025


In [23]:
print(round(model_automl.leader.auc(valid=True),n_digits))

0.9597


In [26]:
print(model_automl.leader.confusion_matrix(valid=True))


Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4804601428484676: 


,,0,1,Error,Rate
0,0,784745.0,64798.0,0.0763,(64798.0/849543.0)
1,1,47541.0,282378.0,0.1441,(47541.0/329919.0)
2,Total,832286.0,347176.0,0.0952,(112339.0/1179462.0)


In [28]:
ss.stop()